In [1]:
import scipy.io as scio
import os
import nibabel as nb
import numpy as np

## 获取类别索引信息

In [2]:
categoryIndicesPath = "stimulus/category_indices.mat"
categoryIndices = scio.loadmat(categoryIndicesPath)

In [3]:
categoryIndices

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Fri Jul 26 11:53:39 2019',
 '__version__': '1.0',
 '__globals__': [],
 'animals': array([[ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
         17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]], dtype=uint8),
 'faces': array([[125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137,
         138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150,
         151, 152, 153, 154, 155, 156]], dtype=uint8),
 'objects': array([[29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44,
         45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60,
         61, 62, 63, 64]], dtype=uint8),
 'people': array([[101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113,
         114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124]],
       dtype=uint8),
 'scenes': array([[ 65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
          78,  79,  80,  81,  82,  83, 

## 获取数据的边缘信息，也就是非零边界

In [4]:
def get_range(data_path):
    img = nb.load(data_path)
    img_data = img.get_fdata()
    img_data[np.isnan(img_data)] = 0
    img_non_zero = np.nonzero(img_data)
    print(len(img_non_zero[0]))
    min_x = min(img_non_zero[0])
    max_x = max(img_non_zero[0])
    min_y = min(img_non_zero[1])
    max_y = max(img_non_zero[1])
    min_z = min(img_non_zero[2])
    max_z = max(img_non_zero[2])
    return min_x, max_x, min_y, max_y, min_z, max_z

In [5]:
min_x, max_x, min_y, max_y, min_z, max_z = get_range("fMRI_data/subj01/beta_0002.nii")
print(min_x, max_x, min_y, max_y, min_z, max_z)

53751
1 49 3 61 4 51


## 稀疏范围

In [6]:
data_path = "fMRI_data/subj01/beta_0001.nii"
def sparse_generate(data_path):
    img = nb.load(data_path)
    img_data = img.get_fdata()
    data_exist = np.zeros(img_data.shape)
    data_exist[~np.isnan(img_data)] = 1
    dimx_sp = np.sum(np.sum(data_exist,axis=0),axis=0)
    dimy_sp = np.sum(np.sum(data_exist,axis=1),axis=1)
    dimz_sp = np.sum(np.sum(data_exist,axis=2),axis=0)
    dimx_thre = int(max(dimx_sp)*0.95)
    dimy_thre = int(max(dimy_sp)*0.95)
    dimz_thre = int(max(dimz_sp)*0.95)
    return dimy_sp>dimy_thre, dimz_sp>dimz_thre, dimx_sp>dimx_thre
y_sp_index, z_sp_index, x_sp_index = sparse_generate(data_path)
print(y_sp_index, z_sp_index, x_sp_index)

[False False False False False False False False False False False False
 False False False False False False False  True  True False  True False
 False False False False  True  True  True False False  True  True  True
  True False False False False False False False False False False False
 False False False False False] [False False False False False False False False False False False False
 False False False False False False False False False False False False
  True  True  True  True  True  True  True  True  True  True  True  True
  True False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False] [False False False False False False False False False False False False
 False False False False False False False False False False False False
  True  True  True  True  True  True  True  True  True False False False
 False False False False False False False False False False False False

## 读取某一个subject的数据

In [7]:
def get_sub_data(data_path, y_sp_index, z_sp_index, x_sp_index):
    data = []
    for filename in os.listdir(data_path):
        img = nb.load(data_path + '/' + filename)
        img_affine = img.affine
        img_data = img.get_fdata()
        sp_y = img_data[y_sp_index,:,:]
        sp_yz = sp_y[:,z_sp_index,:]
        spy_yzx = sp_yz[:,:,x_sp_index]
        img_data = spy_yzx
        # img_data = img_data.transpose(2,0,1)
        img_data_vec = img_data[~np.isnan(img_data)]
        print(img_data_vec.shape)
        data.append(img_data_vec)
        nb.Nifti1Image(img_data, img_affine).to_filename('fMRI_data/sub01_processed/'+filename)
        # break
    return data

In [8]:
sub_data = get_sub_data('fMRI_data/subj01', y_sp_index, z_sp_index, x_sp_index)
# sub_data = np.array(sub_data, dtype=np.object)
len(sub_data)

(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)
(1084,)


156

## 获取subject的meta信息

In [9]:
# def get_sub_meta(path):
#     img = nb.load(path)
#     img_data = img.get_fdata()
#     # img_data = img_data.transpose(2,0,1)
#     shape = img.shape
#     nvoxels = np.sum(~np.isnan(img_data))
#     indices = np.argwhere(~np.isnan(img_data))
#     coord2Col = np.zeros(img_data.shape)
#     count = 1
#     for induce in indices:
#         coord2Col[induce[0]][induce[1]][induce[2]] = count
#         count += 1
#     meta = {
#         'coordToCol': coord2Col,
#         'colToCoord': indices,
#         'nvoxels': nvoxels,
#         'dimx': shape[0],
#         'dimy': shape[1],
#         'dimz': shape[2],
#         'ntrials': len(sub_data),
#     }
#     return meta

def get_sub_meta(path, y_sp_index, z_sp_index, x_sp_index):
    img = nb.load(path)
    img_data = img.get_fdata()
    sp_y = img_data[y_sp_index,:,:]
    sp_yz = sp_y[:,z_sp_index,:]
    spy_yzx = sp_yz[:,:,x_sp_index]
    img_data = spy_yzx
    # img_data = img_data.transpose(2,0,1)
    shape = img_data.shape
    nvoxels = np.sum(~np.isnan(img_data))
    indices = np.argwhere(~np.isnan(img_data))
    coord2Col = np.zeros(img_data.shape)
    count = 1
    for induce in indices:
        coord2Col[induce[0]][induce[1]][induce[2]] = count
        count += 1
    meta = {
        'coordToCol': coord2Col,
        'colToCoord': indices+1,
        'nvoxels': nvoxels,
        'dimx': shape[0],
        'dimy': shape[1],
        'dimz': shape[2],
        'ntrials': len(sub_data),
    }
    return meta

In [10]:
sub_meta = get_sub_meta("fMRI_data/subj01/beta_0002.nii", y_sp_index, z_sp_index, x_sp_index)

## 保存成mat并储存

In [11]:
subj01_data = {
    'data': sub_data,
    'meta': sub_meta,
    'info': categoryIndices,
}
subj01_data

{'data': [array([-2.33410025, -3.02996612, -2.78727841, ..., -0.23754211,
          1.91275096,  3.20834804]),
  array([-0.38022003,  0.09410933, -4.87061214, ...,  0.84462351,
          0.77601546,  1.80674314]),
  array([ 0.0369109 ,  0.19662905, -2.74461126, ..., -2.06829882,
         -2.18306565, -1.38071477]),
  array([-0.32625723, -2.70232511,  0.87882227, ...,  0.81258988,
          2.09708619,  1.65783179]),
  array([ 1.74758303, -0.562217  , -8.69533539, ..., -2.06279826,
         -0.90687883, -1.14975739]),
  array([-2.55027056, -0.32516599, 10.5340786 , ..., -2.96730447,
         -2.89265943, -2.04849267]),
  array([-0.29614165,  3.58272004,  6.34693289, ..., -2.0824163 ,
          2.25409746,  3.65136433]),
  array([-0.58299637,  3.17713714,  9.9285841 , ...,  2.68049979,
          0.92210078,  1.14200914]),
  array([ 1.78975248, -0.41265589, -4.68054152, ..., -0.83984673,
          0.09091899, -0.57948381]),
  array([ -1.26000094,  -1.04575372, -11.25151825, ...,  -0.81301

In [12]:
scio.savemat('subj01-data.mat', mdict=subj01_data)